# Generating text with Recurrent Neural Networks

## Data Reading
Load the file from the data folder and inspect it. Standardize to lowercase. How long is the corpus?

In [ ]:
import os


datasets_path = os.path.join(os.path.dirname(os.getcwd()), 'datasets')
musquetaires_path = os.path.join(datasets_path, 'musquetaires')

filename = os.path.join(musquetaires_path, 'musquetairesShort')
raw_text = open(filename).read()

In [ ]:
raw_text[:100]

In [8]:
text = raw_text.lower()
print('corpus length:', len(text))

corpus length: 198059


### Text preparation
We create a set with the different characters and two dictionaries from indices to chars
<font color=red><b>Generate dictionaries for the char to indices and indices to chars.
<br>_Hint: use the enumerate function on the chars set_</b>
</font>

In [9]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 52


Next we generate the input and output arrays:

The input will consist on sentences of a fixed (_maxlen_) lenght, while the outputs will be the next characters in the text.

So, if the text is "Welcome to Big Data Spain" with _maxlen_ = 5, we will have:


In order to avoid overfitting (and improve performances) we can add a _step_ to the structure so that with step = 3, for example:

<font color=red><b>Fill the sentences and next_char lists with the input and output data</b></font>

In [10]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 66007


In [11]:
sentences[:5]

['author’s preface\n\n\nin which it is proved',
 'hor’s preface\n\n\nin which it is proved th',
 '’s preface\n\n\nin which it is proved that,',
 'preface\n\n\nin which it is proved that, no',
 'face\n\n\nin which it is proved that, notwi']

In [12]:
next_chars[:5]

[' ', 'a', ' ', 't', 't']

### Dataset generation
We turn the text into one-hot-like vectors. Initialize the Input and output arrays to zero as boolean

In [13]:
import numpy as np
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
Y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    Y[i, char_indices[next_chars[i]]] = 1

In [14]:
X[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [15]:
print ("timesteps = ", len (X[0]), ", numchars = ", len (X[0][0]))

timesteps =  40 , numchars =  52


## Model Generation
Build the LSTM model to be trained train on the data, on this config:
- LSTM layer, with 256 units
- LSTM layer, with 256 units
- Dense layer, with 64 units
- Dense softmax layer
- On compilation, use adam as the optimizer and categorical_crossentropy as the loss function.
- Print the summary


<font color=red><b>Remember to initialize it propperly and to include input_shape on the first layer. <br> Hints: input_shape= (maxlen, len(chars))
- Use the imported libraries</b></font>

In [18]:
import os
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.clear_session() 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM

In [19]:

model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(LSTM(256))
model.add(Dense(64))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 256)           316416    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 52)                3380      
_________________________________________________________________
activation (Activation)      (None, 52)                0         
Total params: 861,556
Trainable params: 861,556
Non-trainable params: 0
_________________________________________________________________


### Model Training
Train the model for a couple of epochs and see how it works. Use a batch_size of 128

In [ ]:
model.fit(X, Y,
      batch_size=128,
      epochs=2)

Train on 66007 samples
Epoch 1/2
 2432/66007 [>.............................] - ETA: 7:38 - loss: 3.4267

### Model Evaluation
Let's test our model. In order to obtain a probabilistic answer we can sample from a probability array instead of just taking the max argument:

<font color=red><b> Sometimes probabilities are rounded. Apply a normalization-like tratment to them in order to avoid this when sampling</b> </font>


$$ p_i = \frac{p_i}{\sum_j p_j}$$


In [ ]:
def sample(preds, sample = True):
    if sample:
        # probs can be rounded and not sum up to one. We recalculate in order to avoid this
        preds = np.asarray(preds).astype('float64')
        preds = preds /np.sum (preds)
        probas = np.random.multinomial(1, preds, 1)
    else:
        probas = preds
    return np.argmax(probas)

We get a seed in order to predict:

In [ ]:
import random
start_index = random.randint(0, len(text) - maxlen - 1)
generated = ''
sentence = text[start_index: start_index + maxlen]
generated += sentence
print (generated)

#### Predictions
This will be the secuence for which we are going to predict the next character:

<font color=red> <b> Predict the next character given the input x_pred. <br>Hint: remember to take the first item in list</b>  </font>

In [ ]:
## Predict next character given a model and the sequence to predict 
def get_next_char (model, x_pred, indices_char, Sample = True):
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, 1.0)
    return indices_char[next_index]

In [ ]:
x_pred = np.zeros((1, maxlen, len(chars)))
for t, char in enumerate(sentence):
    x_pred[0, t, char_indices[char]] = 1.

Sample = True
    
next_char = get_next_char (model, x_pred, indices_char, Sample)  

print (next_char)

Let's predict some more characters:

In [ ]:
import sys
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]
print('Seed: ' + sentence + '"')
print('---------------------- Generated Text -----------------------')
# sys.stdout.write(generated)
for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    next_char = get_next_char (model, x_pred, indices_char, Sample)
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()

## Load a trained model
Training Deep Learning Models is time consuming. So, some pretrained models are available to be loaded and take a look at better predictions. We will load a model for each 5 epochs in order to see the evolution. 

<font color=red> <b> Load a model for each time and predict the text <br> Hint: You can load the whole model or just the weights as the configuration is the same</b>  </font>

In [ ]:
count = 0
partial_n_epoch = 5
times = 12

## Set both starts for the seed sentence and the multinomial prediction
np.random.seed (2)
for j in range (times):
    
    count += partial_n_epoch
    print ("")
    print ("-------------- Next Model --------------")
    print ("Trained on ", count, " epochs")
    modelName = os.path.join(musquetaires_path,'MusquetairesModelOptimizedMode_' + str (count) + '.h5' )

    model.load_weights (modelName)
   
    start_index = np.random.randint(0, len(text) - maxlen - 1)
    sentence = text[start_index: start_index + maxlen]
    print('Seed: ' + sentence + '"')
    print('---------------------- Generated Text -----------------------')
    # sys.stdout.write(generated)
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        next_char = get_next_char (model, x_pred, indices_char, Sample)
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
